In [7]:
# 불균형 데이터 생성(1:9) 악성:양성
from sklearn.datasets import load_breast_cancer
import numpy as np
from sklearn.pipeline import Pipeline
np.random.seed(42)

data = load_breast_cancer()
X,y = data.data, data.target
# 악성을 소수 클래스로 생성

print(f'악성 양성의 오리지널 비율 : {np.unique(y, return_counts = True)}')
m_index = np.where(y ==0)[0] #where는 0이 되는 인덱스(악성)를 반환
b_index = np.where(y ==1)[0] 

# 악성은 일부만, 양성은 더 많이 사용
# 악성의 30%만 사용, 양성은 전체 1:9
size_30 = int(len(m_index)*0.3)
selected_m_index = np.random.choice(m_index,size=size_30, replace=False)
selected_b_index = b_index

# 섞어주기
concatenate_selected_index = np.concatenate([selected_m_index, selected_b_index])
np.random.shuffle(concatenate_selected_index)

X_imb = X[concatenate_selected_index]
y_imb = y[concatenate_selected_index]

# 클래스 분포 확인
unique,counts = np.unique(y_imb, return_counts = True)
print('클래스 분포 : ')
for label, count in zip(unique, counts):
    percentage = count / len(y_imb) * 100
    print(f'클래스 {label} : {count}개 ({percentage:.1f}%)')

# 불균형을 무시하고 모델의 성능 평가 진행
# 스케일링 정규화 StandardScaler
# LogisticRegression
# pipe
# 평가는 class report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
x_train, x_test, y_train, y_test = train_test_split(X_imb, y_imb, stratify=y_imb, test_size=0.2, random_state=42)
print('1. 기본 모델 (불균형 무시)')
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(random_state=42, max_iter=1000))
])
pipe.fit(x_train,y_train)
print( classification_report(y_test, pipe.predict(x_test), target_names=['양성', '악성']) )


악성 양성의 오리지널 비율 : (array([0, 1]), array([212, 357]))
클래스 분포 : 
클래스 0 : 63개 (15.0%)
클래스 1 : 357개 (85.0%)
1. 기본 모델 (불균형 무시)
              precision    recall  f1-score   support

          양성       1.00      1.00      1.00        13
          악성       1.00      1.00      1.00        71

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84



In [8]:
print('불균형 해결 : 클래스 가중치 사용')
# 기존 파이프라인의 clf 이름의 객체 파라메터를 조정
from copy import deepcopy
#pipe_weight = deepcopy(pipe)
#pipe.set_params(clf__class_weight = 'balanced')

pipe_weight = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'))
])

pipe_weight.fit(x_train, y_train)
print( classification_report(y_test, pipe_weight.predict(x_test)))

print("가중치 계산")
n_samples = len(y_train)
n_classes = 2
class_counts = np.bincount(y_train)
for i in range(n_classes):
    weight = n_samples / (n_classes * class_counts[i])
    print(f'클래스 : {i} : {weight:.3f}')
    

불균형 해결 : 클래스 가중치 사용
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        71

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84

가중치 계산
클래스 : 0 : 3.360
클래스 : 1 : 0.587


In [ ]:
from sklearn.ensemble import RandomForestClassifier
print('불균형 : RandsomFores (균형모드)')
pipe_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(class_weight='balanced', random_state=42))
])
pipe_rf.fit(x_train, y_train)
y_pred = pipe_rf.predict(x_test)
print( classification_report(y_test, y_pred))

불균형 : RandsomFores (균형모드)
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       1.00      0.99      0.99        71

    accuracy                           0.99        84
   macro avg       0.96      0.99      0.98        84
weighted avg       0.99      0.99      0.99        84



In [ ]:
# 오버 or 언더 샘플링 - 직접 실행 SMOTE, SMOTEEN 둘다 써보기
